
# ETL

In [1]:
import pandas as pd
from textblob import TextBlob

FUNCION PARRA ARREGLAR LOS JSON CORRUPTOS

-----------------------------------
## ETL STEAM_GAMES

In [2]:
# Leemos y creamos el dataframe steam_games
steam_games = pd.read_parquet("../Datasets/steam_games_arreglado.parquet")


Borramos filas nulas:

In [3]:
steam_games.dropna(inplace=True)

Borramos columnas que no vamos a usar:

In [4]:
# Borramos las columnas inecesarias
columns_delete_games = ["publisher", "title", "url", "tags", "reviews_url", "specs", "early_access"]
steam_games.drop(columns=columns_delete_games, axis=1, inplace=True)

Renombramos las columnas para asemejar con las otras base de datos:

In [5]:
# Modificamos los nombres de la columna id = item_id, app_name = item_name para futuros merge con los otros datasets
steam_games.rename(columns = {"id": "item_id","app_name" : "item_name"}, inplace = True)

Desanidamos la columna genres:

In [6]:
steam_games = steam_games.explode(column='genres')

Transformamos la columna release_date para quedarnos con el año solo y llenamos los valores nulos con Unknown:

In [7]:
# Establecemos la columna year paara los años y la convertimos en int y los nulos en 0 
steam_games["release_date"] = pd.to_datetime(steam_games['release_date'], errors='coerce') #Convertimos en la columna release_date a valores date

steam_games['release_year'] = steam_games['release_date'].dt.year.fillna(-1).astype(int) #  ceamos la columna release_year con los años de la columna release_date y los nulos son 'Unknown'
 
steam_games['release_year'] = steam_games['release_year'].astype(str).replace('-1','Unknown') # Comvertimos finalmente en string y las fechas que eran nulas en Unknown(Desconocido)

# Borramos la columna vieja 'release_date'
steam_games.drop(columns='release_date', inplace=True)

Convertimos los valores en entero los valores de la columna 'item_id'

In [8]:
steam_games["item_id"] = steam_games["item_id"].astype(int)

Convertimos en float los valores de la columna  price:

In [9]:
steam_games['price'] = steam_games['price'].astype(float)

Borramos duplicados:

In [10]:
steam_games.drop_duplicates(inplace=True)

Reseteamos el indice y ordenamos segun la columna item_id:

In [11]:
steam_games.sort_values(by="item_id", inplace=True)

steam_games.reset_index(inplace=True, drop=True)

Guardamos dataset final:

In [12]:
steam_games.to_parquet("../Datasets_final/steam_games_final.parquet.gz", compression = "gzip", index=False)



-------------------------------------
## TRABAJAMOS CON  ARCHIVO USER_REVIEWS

In [13]:
user_reviews = pd.read_parquet('../Datasets/user_reviews_arreglado.parquet') 

Desanidamos la columna 'reviews':

In [14]:
df_duplicados = user_reviews.explode(column= "reviews") # creamos un dataframe donde duplicamos las filas de acuerdo a la cantidad de subcampos que hay en la columna review.    

# BORRAMOS LAS FILAS NULAS QUE TENGA REVIEW
df_duplicados.dropna(subset="reviews", inplace = True) # Borramos las filas de la columna reviews que sean nulos, ya que es la unica columna que tiene nulos

# RESETEAMOS INDICE PARA CONCATENAR DESPUES
df_duplicados.reset_index(inplace=True, drop=True) # Reseteamos los indices del dataframe desanidado para poder concatenar

In [15]:
# CREAMOS UN DADAFRAME CON LA COLUMNA REVIEWS YA DESANIDADA.

diccionario = { "funny" : [],  # Creamos diccionario donde se almacenan los datos de la columna reviews
                "posted" : [],
                "last_edited" : [],
                "item_id" : [],
                "helpful" : [],
                "recommend": [],
                "review" : []}

for fila in df_duplicados["reviews"]:     # Ieramos la columna para que me queden solo los diccionarios
    for i, items in enumerate(fila.values()):  #Iteramos cada valor del diccionario para que se agregue al diccionario antes creado
        if i == 0:
                diccionario["funny"].append(items)
        elif i == 4:
                diccionario["posted"].append(items)
        elif i ==2:
                diccionario["item_id"].append(items)
        elif i == 3:
                diccionario["last_edited"].append(items)
        elif i == 1:
                diccionario["helpful"].append(items)
        elif i == 5:
                diccionario["recommend"].append(items)
        elif i == 6:
                diccionario["review"].append(items)

df_separado_reviews = pd.DataFrame(diccionario) # Creamos el df de solo los elementos de la columna reviews



Unimos los dos dataframes:

In [16]:
# Concatenamos en un solo dataframe
user_reviews_final = pd.concat([df_duplicados,df_separado_reviews],axis=1) 

AGREGAR COLUMNA DE ANALISIS DE SENTIMIENTO AL DATAFRAME

In [17]:
# CREAMOS LA FUNCION PARA REALIZAR UN ANÁLISIS DE SENTIMIENTO
from textblob import TextBlob # Usamos la libreria textblob para ello

def analisis_de_sentimiento(cadena_caracteres):
    """Esta funcion devuelve un valor segun el analisis de sentimiento que realize al strig. Como parametro recibe un str y puede devolver (0: Malo, 1: Neutral, 2: Positivo )"""
    
    analizador = TextBlob(str(cadena_caracteres)) # La libreria Texplot permite hacer un analisis de sentimiento sobre un string
    
    if analizador.sentiment.polarity > 0.1: 
        return 2  
    elif analizador.sentiment.polarity < -0.1:
        return 0  
    else:
        return 1 
    

# Aplicamos la funcion a la columna review
user_reviews_final["sentiment_analysis"] = user_reviews_final["review"].apply(analisis_de_sentimiento) 

# Comprobamos el analisis de sentimiento
user_reviews_final.sentiment_analysis.value_counts() 

sentiment_analysis
2    26263
1    24959
0     8083
Name: count, dtype: int64

Borramos columnas innecesarias:

In [18]:
# BORRAMOS LAS COLUMNAS QUE NO VAMOS A NECESITAR:
columnas_borrar_reviews = ["user_url", "funny", "posted", "last_edited", "helpful", "reviews", "review"] # Establecemos las columnas a borrar
user_reviews_final.drop(columns=columnas_borrar_reviews, inplace=True, axis=1) # Borramos esas columnas

Convertimos la columna item_id a entero:

In [19]:
# Convertir la columna item_id a tipo entero
user_reviews_final['item_id'] = user_reviews_final['item_id'].astype(int)

Borramos duplicados:

In [20]:
user_reviews_final.drop_duplicates(inplace = True)

Ordenamos y reseteamos indice:

In [21]:
user_reviews_final.sort_values(by="item_id", inplace=True) #Ordenamos por item_id

user_reviews_final.reset_index(inplace=True, drop=True) # Reseteamos index


Guardamos datasets final:

In [22]:
user_reviews_final.to_parquet("../Datasets_final/user_reviews_final.parquet.gz", compression = "gzip", index=False)

--------------------------------------------
## TRABAJAMOS CON  ARCHIVO USER_ITEMS

In [23]:
user_items = pd.read_parquet('../Datasets/user_items_arreglado.parquet')

Desanidamos columna items:

In [24]:
df_duplicados2 = user_items.explode(column= "items") # creamos dataframe donde duplica los campos de los usuarios de acuerdo a la cantidad de campos de reviews o items que haya en esa calumna por usuario .  

df_duplicados2.dropna(subset='items', inplace = True)

In [25]:
# CREAMOS UN DATAFRAME LA COLUMNA DF DESANIDADA

diccionario2 = { "item_id" : [],  # Creamos diccionario donde se almacenan los datos que estan en los diccionarios
                "item_name" : [],
                "playtime_forever" : [],
                "playtime_2weeks" : []}

for fila in df_duplicados2["items"]:     # Ieramos la columna para que me queden solo los diccionarios
    for i, items in enumerate(fila.values()):  #Iteramos cada valor del diccionario para que se agregue al diccionario antes creado
        if i == 0:
                diccionario2["item_id"].append(items)
        elif i == 1:
                diccionario2["item_name"].append(items)
        elif i ==3:
                diccionario2["playtime_forever"].append(items)
        elif i == 2:
                diccionario2["playtime_2weeks"].append(items)


df_separados_items = pd.DataFrame(diccionario2) # Creamos el df de la columna items

In [26]:
# Reseteamos el indice para concatenar
df_duplicados2.reset_index(inplace=True, drop=True) # Reseteamos ya que de lo contrario no nos dejara concatenar

In [27]:
# Concatenamos
user_items_final = pd.concat([df_duplicados2,df_separados_items],axis=1) # Concatenamos los dos dataframes

Borramos columnas que no necesitamos:

In [28]:
# Borramos columnas que no usemos
columnas_borrar_items = ["steam_id","items", "user_url","items","playtime_2weeks"]
user_items_final.drop(columns=columnas_borrar_items, inplace = True) # Borramos las columnas

Transformamos item_id en INT:

In [29]:
# TRANSFORMAMOS LA COLUMNA item_id

user_items_final["item_id"] = user_items_final["item_id"].astype(int) # Convertimos en int

Borramos duplicados:

In [30]:
user_items_final.drop_duplicates(inplace=True) # Borramos filas duplicadas

Ordenamos y reseteamos indices:

In [31]:
user_items_final.sort_values(by="item_id",inplace=True) # Ordenamos por item_id

user_items_final.reset_index(inplace=True, drop=True) # Reseteamos index

Guardamos datasets final:

In [32]:
user_items_final.to_parquet("../Datasets_final/user_items_final.parquet.gz",index=False, compression = "gzip")